# IN OUT machine learning model

In [20]:
import os
from utils.pipeline import get_folders,folder_analysis
import pandas as pd
import numpy as np

folder_name='imgs_conce'

list_folders = get_folders(folder_name)

base_path = os.path.dirname(list_folders[0])

_,_,_,_,result = folder_analysis(list_folders)


df = pd.read_csv('conce_bbox.csv')


# Convert result['In'] to integers
result_in_int = [int(id) for id in result['In']]

df['direction'] = np.where(df['id'].isin(result_in_int), 'In', 'Out')
df.to_csv('conce_bbox.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Load your CSV data into a DataFrame
# Assuming your CSV file is named 'coordinates.csv'
df = pd.read_csv('conce_bbox_WIP.csv')

# Calculate the mean of the coordinates for each ID
mean_coordinates = df.groupby('id').mean().reset_index()

# Features to be used for clustering
features = ['x1', 'y1', 'x2', 'y2', 'centroid_bottom_x', 'centroid_bottom_y']

# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(mean_coordinates[features])

# Perform K-Means clustering with 3 clusters
kmeans = KMeans(n_clusters=3, random_state=42)
mean_coordinates['cluster'] = kmeans.fit_predict(scaled_features)

# Now mean_coordinates DataFrame contains an additional column 'cluster' 
# indicating the cluster assignment (0, 1, or 2) for each ID
print(mean_coordinates[['id', 'cluster']])


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

# Load your data
df = pd.read_csv('your_data.csv')

# Assuming 'direction' is your target variable
X = df[['x1', 'y1', 'x2', 'y2', 'centroid_bottom_x', 'centroid_bottom_y']]
y = df['direction']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Initialize the XGBoost classifier
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Train the model
model.fit(X_train, y_train)



# Making predictions
y_pred_proba = model.predict_proba(X_test)
y_pred = model.predict(X_test)

# Define a threshold for maximum probability to decide 'None'
threshold = 0.5  # This can be adjusted based on validation

# Apply threshold to probabilities to assign 'None' to uncertain predictions
y_pred_final = [y_pred[i] if max(probs) > threshold else 'None' for i, probs in enumerate(y_pred_proba)]

# Evaluate the model
print(classification_report(y_test, y_pred_final))
